# GPU VRAM 계산


### 1. GPU VRAM 계산 추천 사이트

다음 사이트들은 모델의 파라미터 수와 정밀도(Precision)를 기반으로 필요한 메모리를 자동으로 추산해 준다.

- **Hugging Face Model Memory Usage (공식 Space)**
    - https://huggingface.co/spaces/hf-accelerate/model-memory-usage
- **LLM-VRAM-Calculator**
    - https://llm-vram-calc.titanium-monkey.com/
    - https://vram.asmirnov.xyz/
    - 파라미터 수, 양자화(Quantization) 수준, 컨텍스트 길이(Context Window) 등을 상세하게 설정하여 계산할 수 있다. 학습 시 Optimizer(AdamW 등)에 따른 추가 메모리까지 계산해 주어 유용하다.

- **간편 계산:** 파라미터 수(B)에 **2를 곱하면 FP16 용량**, **0.5를 곱하면 INT4 용량**이 나온다. (단위: GB)
- **안전 마진:** 계산된 용량의 1.2배 이상 되는 GPU를 선택하는 것이 좋다.

### 2. GPU VRAM 직접 계산 방법 (공식)

sLLM 구동에 필요한 VRAM은 크게 **모델 가중치(Weights)**, **KV 캐시(KV Cache)**, **활성화(Activation)** 메모리로 구성된다. 가장 큰 비중을 차지하는 것은 모델 가중치이다.

### 기본 공식 (모델 가중치)

모델의 파라미터 수(Parameter Count)와 데이터 정밀도(Precision)를 곱하여 계산한다.

$$\text{VRAM (GB)} \approx \text{Parameters (Billion)} \times \text{Bytes per Parameter}$$

**정밀도(Precision)별 바이트 수:**

| 데이터 타입 | 비트 수 | 파라미터당 바이트 | 설명 |
| :--- | :--- | :--- | :--- |
| **FP32** (Full Precision) | 32-bit | 4 Bytes | 일반적인 학습 시 사용 |
| **FP16 / BF16** | 16-bit | 2 Bytes | 대부분의 추론 및 LoRA 학습 |
| **INT8** | 8-bit | 1 Byte | 기본적인 양자화 |
| **INT4** (GPTQ/AWQ) | 4-bit | 0.5 Bytes | 로컬 구동 시 가장 많이 사용 |

### 실전 계산 예시: 8B (80억) 모델

**1) FP16 (16-bit) 원본 모델 구동 시:**

$$8 (\text{Billion}) \times 2 (\text{Bytes}) = 16 \text{GB}$$

> 결과: 모델 로드에만 약 16GB가 필요하며, 컨텍스트 처리를 위해 24GB VRAM GPU(RTX 3090/4090 등)가 권장된다.
>

**2) INT4 (4-bit) 양자화 모델 구동 시:**

$$8 (\text{Billion}) \times 0.5 (\text{Bytes}) = 4 \text{GB}$$

> 결과: 모델 로드에 4GB가 필요하다. 여기에 KV 캐시 여유분을 더하면 6GB~8GB VRAM GPU에서도 구동이 가능하다.
>


### 3. 추가 고려 사항 (Overhead)

단순 모델 크기 외에 실제 구동 시에는 다음과 같은 추가 메모리가 필요하다. 따라서 계산된 값에 약 **20%의 여유 공간**을 두는 것이 안전하다.

1. **KV Cache (컨텍스트 길이):** 입력 프롬프트와 생성할 토큰의 길이가 길어질수록 메모리 사용량이 선형적으로 증가한다. 긴 문맥을 처리하려면 더 많은 VRAM이 필요하다.
2. **Activation Overhead:** 연산 중간에 발생하는 임시 데이터들을 저장할 공간이 필요하다.
3. **라이브러리 오버헤드:** PyTorch, CUDA 커널 등이 사용하는 기본 메모리가 있다 (약 0.5GB ~ 1GB).


In [1]:
%pip install -Uq transformers datasets accelerate trl peft bitsandbytes hf_transfer wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 133.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 19.2 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Thu Feb 12 05:37:55 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# gpu 메모리 조회
import torch

print(torch.cuda.memory_allocated())  # 현재 사용중인 메모리
print(torch.cuda.memory_reserved())   # 사용중 + 재사용을 위한 cache메모리

0
0


In [ ]:
# gpu 메모리 조회
import torch

print(f'할당 vram: {torch.cuda.memory_allocated() / 1024 ** 3: .2f}GB')  # 현재 텐서가 실제로 사용중인 메모리(GB)
print(f'캐싱 vram: {torch.cuda.memory_reserved() / 1024 ** 3: .2f}GB')   # Pytorch가 재사용을 위해 확보해둔 메모리(GB)


할당 vram:  0.00GB
캐싱 vram:  0.00GB


In [7]:
# 현재 GPU VRAM 사용량을 출력하는 함수
def print_gpu_utilization():
    if torch.cuda.is_available():  # GPU 가용시
        print(f'GPU VRAM 사용량: {torch.cuda.memory_allocated() / 1024 ** 3: .2f}GB')
    else:
        print('Runtime 유형을 GPU로 변경하세요.')

print_gpu_utilization()

GPU VRAM 사용량:  2.38GB


In [8]:
# 모델 로드 및 gpu 확인
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, dtype='auto', device_map='auto')
    print_gpu_utilization()
    return model, tokenizer

model_id = 'LGAI-EXAONE/EXAONE-4.0-1.2B'
model, tokenizer = load_model_and_tokenizer(model_id)

Loading weights:   0%|          | 0/332 [00:00<?, ?it/s]

GPU VRAM 사용량:  4.77GB
